In [2]:
import json
import pandas as pd
df = pd.read_json('D:/e/SEM4/NLP/Musical_Instruments_5.json',lines = True)
df

ValueError: Expected object or value

reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
asin - ID of the product, e.g. 0000013714
reviewerName - name of the reviewer
helpful - helpfulness rating of the review, e.g. 2/3
reviewText - text of the review
overall - rating of the product
summary - summary of the review
unixReviewTime - time of the review (unix time)
reviewTime - time of the review (raw)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.describe().transpose()

In [ ]:
print("\n\n\nMissing Values:\n",df.isna().sum())

In [ ]:
#Total number of products
df.asin.nunique()

In [ ]:
#Total number of unique users
df.reviewerID.nunique()

a. Counts, averages

In [ ]:
df.count()

In [ ]:
df.mean()

b. Distribution of the number of reviews across products

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df)

In [ ]:
ratings= df['overall'].value_counts().keys().tolist()
values = df['overall'].value_counts().tolist()

In [ ]:
print(f'Rating   Distribution')
for i in range(len(ratings)):
    print(f'{ratings[i]}      {values[i]}')

In [ ]:
num_bin=100
plt.hist(df['overall'],num_bin,alpha=0.5)
plt.title('Product Rating')
plt.xlabel("Rating")
plt.ylabel("Products")
plt.show()

c. Distribution of the number of reviews per product

In [ ]:
products= df['asin'].value_counts().keys().tolist()
reviewsPerProducts = df['asin'].value_counts().tolist()

In [ ]:
print(f'Products   Reviews count')
for i in range(len(products)):
    print(f'{products[i]}      {reviewsPerProducts[i]}')

In [ ]:
highestRatedProduct=df.loc[df['asin'] == "B003VWJ2K8"]
highestRatedProduct["overall"].value_counts()

In [ ]:
lowestRatedProduct=df.loc[df['asin'] == "B004HJ9COU"]
lowestRatedProduct["overall"].value_counts()

In [ ]:
plt.hist(df['asin'].value_counts(),num_bin,alpha=0.5)
plt.xlabel(' Reviews')
plt.ylabel('Products')
plt.title('Distribution of Reviews per product')
plt.show()

In [ ]:
user= df['reviewerID'].value_counts().keys().tolist()
reviewsPerUser = df['reviewerID'].value_counts().tolist()

In [ ]:
print(f'Users:Reviews count')
for i in range(len(user)):
    print(f'{user[i]}:{reviewsPerUser[i]}')

In [ ]:
plt.hist(df['reviewerID'].value_counts(),num_bin,alpha=0.5)
plt.xlabel('Reviews')
plt.ylabel('Users')
plt.title('Distribution of Reviews per User')
plt.show()

# 2. Text basic pre-processing:

In [ ]:
#Stratified sampling
random_sample = df.groupby('overall', group_keys=False).apply(lambda x: x.sample(200))


In [ ]:
random_sample['overall'].value_counts()

In [ ]:
def product_ratings(x):
    if x['overall'] == 5 or x['overall'] == 4:
        x['ratings'] = 'Positive'
    elif x['overall'] == 3:
        x['ratings'] = 'Neutral'
    elif x['overall'] == 2 or x['overall'] == 1:
        x['ratings'] = 'Negative'
    return x

In [ ]:
random_sample = random_sample.apply(product_ratings, axis = 1)


In [ ]:
random_sample['ratings'].value_counts()

In [ ]:
random_sample.info()

In [1]:
random_sample.drop(['reviewerName','helpful','unixReviewTime','reviewTime'], axis=1, inplace= True)

NameError: name 'random_sample' is not defined

In [29]:
random_sample.columns

Index(['reviewerID', 'asin', 'reviewText', 'overall', 'summary', 'ratings'], dtype='object')

## Data Cleanup

In [30]:
'''Lowercasing'''

#random_sample['reviewText1'] = random_sample['reviewText'].apply(lambda x: ["".join(i.lower()) for i in x.split(' ')])

random_sample['reviewText'] = random_sample['reviewText'].str.lower()

In [31]:
random_sample.head(500)

,reviewerID,asin,reviewText,overall,summary,ratings
5239,A2P2ST28QAZMJI,B000OY4WDI,"i bought this pedal new, hoping to get an elec...",1,The engineers at Boss should be ashamed of the...,Negative
5771,A1803TC7DCT5ZA,B000VJJQUU,"be careful; i have a small recording studio, s...",1,"Ordered three of these, EVERY SINGLE ONE broke...",Negative
8631,A1F4KCBVDP8S25,B004ISK48S,"i purchased this amp locally and initially, i ...",1,Fizz and noise issues,Negative
5035,A35XRT4BW4I6UD,B000L7MNUM,not very bright. not very flexible. too big. c...,1,Bummer,Negative
7850,A3K9OQPCI8UJE,B003BFYDBS,one of the ball bearings designed to catch and...,1,Faulty Product,Negative
...,...,...,...,...,...,...
1360,A6GXP9ESXIH7O,B0002E1G5C,this winder and cutter...winds and cuts fine.....,3,Good but not excellent...,Neutral
9067,A3LNNEYLGGCO25,B005CX4GLE,"i've got three of these to hold an acoustic, a...",3,Ok but a bit flimsier than expected.,Neutral
506,A24JUT24W1K3CV,B0002CZVWS,although this capo is marked as correct for a ...,3,Not Enough Pressure For A Mandolin,Neutral
9212,ADOSJR623FDF1,B005GZ86ZA,"while this hanger is not unattractive, and is ...",3,Stands out,Neutral


In [32]:
'''Remove puncutations'''

random_sample['reviewText'] = random_sample['reviewText'].str.replace('[^\w\s]','', regex=True)

In [33]:
random_sample.head(10)

,reviewerID,asin,reviewText,overall,summary,ratings
5239,A2P2ST28QAZMJI,B000OY4WDI,i bought this pedal new hoping to get an elect...,1,The engineers at Boss should be ashamed of the...,Negative
5771,A1803TC7DCT5ZA,B000VJJQUU,be careful i have a small recording studio so ...,1,"Ordered three of these, EVERY SINGLE ONE broke...",Negative
8631,A1F4KCBVDP8S25,B004ISK48S,i purchased this amp locally and initially i t...,1,Fizz and noise issues,Negative
5035,A35XRT4BW4I6UD,B000L7MNUM,not very bright not very flexible too big clun...,1,Bummer,Negative
7850,A3K9OQPCI8UJE,B003BFYDBS,one of the ball bearings designed to catch and...,1,Faulty Product,Negative
4333,A3I9IGMS9GHW8J,B000BYPJ1U,it doesnt seem to work i think maybe mine was ...,1,I don't get it.,Negative
2325,A1RXH9W4TZB2EF,B0002GLDQM,i use clayton 126mm picks a heavy pick but cer...,1,"Sorry, but...",Negative
3160,A20TFY5J3WNJU2,B0002KZEMC,this pickguard was used it not only was missin...,1,I payed for a new pickguard but got a used one.,Negative
7996,A33PUQETQAM8XT,B003K16N0C,i play harp with some some that i perform i fo...,1,Notes off Hard to play,Negative
223,A3AOB0VF6H0IF4,B000165DSM,received it in time standard blister packaging...,1,DIED after 45 days of use,Negative


In [34]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)

random_sample[['reviewText']].head(10)

<ipython-input-34-1b81a8bb97f6>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,reviewText
5239,i bought this pedal new hoping to get an electric solid body guitar to sound somewhat like an acoustic guitar on stage i knew going in that it would not sound like a martin d28 but i thought it would be good enough to play a couple of acoustic parts without having to switch guitarsimagine my profound disappointmentreally boss this is by far the worst pedal i have ever used of yours this thing has a terrible hiss when you turn the top control higher than about 9 oclock at first i thought i must have a defective unit but then i started reading reviews online seems like it is part of the design its too bad because one can hear some promise in the faux acoustic tones hidden beneath a wall of fizz totally unusable as far as im concerned by the way i tested it direct to a pa as well as into a guitar amp with a single coil guitar strat
5771,be careful i have a small recording studio so we put our cables through a lot and these guys just dont last they were fine for the first few months but then they started to die like a murder mystery seriously one after another they would just go for no apparent reasonfine one second and then it would just go horribly staticky i actually had two blow at one time and thought my pa was shot thankfully it was just these cables not good spend a little extra on another brand these have been my favoritegls audio 25ft cable cords
8631,i purchased this amp locally and initially i thought it was pretty cool as it was my first modeling amp lots of potential for a limitless variety of sounds this was my first fender amp as wellthe presets were neat and all but when i played around with some of the standard models just the amp model no effects just to see how its clean sound was the amp just hissed as if i had the volume cranked with a distortion pedal engaged at max the master volume was at 2 and no effects were turned on ive never heard a clean channel with so much noise behind it even when there was no guitar plugged in all i could hear while playing was the underlying hiss coming through the notes very distracting and it drove me nuts this isnt a ground loop issue my other amps sound just fine at bedroom volumes and beyond its clearly a defect with this product dirty presets with lots of gain and extra volume sound even worse again even when nothing is plugged in and at low master volume youd think i was running a few super high gain stomp boxes with a red hot single coil but nothing was plugged in and all i get is staticyhiss at low levels wtfthe next issue is the weird fizz sound that was especially noticeable on the low e after striking a string instead of a clean note theres a weird fizzy digital artifact like sound that surrounds the note and fades as the string stops moving it almost sounds like cymbals crashing on a poorly authored mp3 file at a low bit rate and frequency whole chords break down and decay in this fizzy envelope and really makes this thing unplayablethe weird thing is neither one of these problems arose with head phones plugged in or when this was connected to my computed via either usb or the line in but im not going to be stuck playing a 100 watt amp through my pc or cans i got the iii instead of the ii because of the power behind it and the touted 12 celestion speakerread the many reviews online or forum posts on many of the guitaramp related websites theres a 65 page forum post on the fender support site that is quite interesting and youll find that these amps are having issues apparently the mustang i and ii dont have these issues and the firmware update did nothing to correct my amp is this a hardware issue software issue speaker issue there is speculation that the lcd screen which is not present on the i or ii may be causing problems but no one seems to know for sure and fender doesnt seem to caredefinitely try this before you buy it you may get an amp that doesnt have these problems you may not notice the sounds or you may not even care let your ears tell you

In [35]:
'''Lemmitization'''

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [36]:
random_sample['reviewText'] = random_sample['reviewText'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


In [37]:
random_sample.head(5)

,reviewerID,asin,reviewText,overall,summary,ratings
5239,A2P2ST28QAZMJI,B000OY4WDI,i bought this pedal new hoping to get an electric solid body guitar to sound somewhat like an acoustic guitar on stage i knew going in that it would not sound like a martin d28 but i thought it would be good enough to play a couple of acoustic part without having to switch guitarsimagine my profound disappointmentreally bos this is by far the worst pedal i have ever used of yours this thing ha a terrible hiss when you turn the top control higher than about 9 oclock at first i thought i must have a defective unit but then i started reading review online seems like it is part of the design it too bad because one can hear some promise in the faux acoustic tone hidden beneath a wall of fizz totally unusable a far a im concerned by the way i tested it direct to a pa a well a into a guitar amp with a single coil guitar strat,1,The engineers at Boss should be ashamed of themselves,Negative
5771,A1803TC7DCT5ZA,B000VJJQUU,be careful i have a small recording studio so we put our cable through a lot and these guy just dont last they were fine for the first few month but then they started to die like a murder mystery seriously one after another they would just go for no apparent reasonfine one second and then it would just go horribly staticky i actually had two blow at one time and thought my pa wa shot thankfully it wa just these cable not good spend a little extra on another brand these have been my favoritegls audio 25ft cable cord,1,"Ordered three of these, EVERY SINGLE ONE broke within the first year",Negative
8631,A1F4KCBVDP8S25,B004ISK48S,i purchased this amp locally and initially i thought it wa pretty cool a it wa my first modeling amp lot of potential for a limitless variety of sound this wa my first fender amp a wellthe presets were neat and all but when i played around with some of the standard model just the amp model no effect just to see how it clean sound wa the amp just hissed a if i had the volume cranked with a distortion pedal engaged at max the master volume wa at 2 and no effect were turned on ive never heard a clean channel with so much noise behind it even when there wa no guitar plugged in all i could hear while playing wa the underlying hiss coming through the note very distracting and it drove me nut this isnt a ground loop issue my other amp sound just fine at bedroom volume and beyond it clearly a defect with this product dirty presets with lot of gain and extra volume sound even worse again even when nothing is plugged in and at low master volume youd think i wa running a few super high gain stomp box with a red hot single coil but nothing wa plugged in and all i get is staticyhiss at low level wtfthe next issue is the weird fizz sound that wa especially noticeable on the low e after striking a string instead of a clean note there a weird fizzy digital artifact like sound that surround the note and fade a the string stop moving it almost sound like cymbal crashing on a poorly authored mp3 file at a low bit rate and frequency whole chord break down and decay in this fizzy envelope and really make this thing unplayablethe weird thing is neither one of these problem arose with head phone plugged in or when this wa connected to my computed via either usb or the line in but im not going to be stuck playing a 100 watt amp through my pc or can i got the iii instead of the ii because of the power behind it and the touted 12 celestion speakerread the many review online or forum post on many of the guitaramp related website there a 65 page forum post on the fender support site that is quite interesting and youll find that these amp are having issue apparently the mustang i and ii dont have these issue and the firmware update did nothing to correct my amp is this a hardware issue software issue speaker issue there is speculation that the lcd screen which is not present on the i or ii may be causing problem but no one seems to know for su

In [38]:
duplicate_random_sample = random_sample.copy()
y = random_sample.pop('ratings').to_frame()
X = random_sample.copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        X, y,stratify=y, test_size=0.3)

In [39]:
y_train['ratings'].value_counts()

Positive    280
Negative    280
Neutral     140
Name: ratings, dtype: int64

In [40]:
y_test['ratings'].value_counts()

Positive    120
Negative    120
Neutral     60 
Name: ratings, dtype: int64

In [41]:
duplicate_random_sample['ratings'].value_counts()

Positive    400
Negative    400
Neutral     200
Name: ratings, dtype: int64

In [44]:
X_train

,reviewerID,asin,reviewText,overall,summary
1477,A3UZITVEOSRRHD,B0002E1O2M,the nanoweb string do stay clean and they stand up but on the negative sidei replaced one string in my set with a nanoweb and noticed right away the feel wa slippery when picking my finger would slip on the nano string rather than pluck i will stick to cleaning my good old standard string,3,NanoWeb stay clean but
4338,A1QN075G0OKBWJ,B000CBE3G4,this pedal work but doesnt have the depth of echo i expected i doesnt even come close to the same sound quality of my old dan echo de1 pedal that certainly explains the difference in price you get what you pay for im using it with a fender eightyfive solid state amp i tried it with other amp and it sound the same if youre looking for a good slap echo effect i would recommend spending more money and buy the dan echo de1 pedal or even the danelectro blt slap echo pedal is le expensive than the dan echo but still sound better than the fab echo pedal the fab echo pedal work and you can hear a definite difference with and without it it just not a good a i expected im sorry i bought it and i dont recommend this pedal there are much better one on the market shop around,3,It works.
5571,A12P4A1OC41KUO,B000SJJCX4,thing work fine easy to understand setting sturdy enough light nice an bright i like ituse with power supply and not batteriesworks best,4,no problems here
4253,A2Z89YMZZJWBHS,B000B6FBA2,i ordered one of these because i like my other planet wave capo a lot i believe it the older metal version of this not the 34lite34 however this one just doe not screw down enough to hold down the string especially down at the lower fret where the neck is a little thinner on a martin jumbo acoustic it will barely bend the string down behind the fret but not enough to prevent buzzing your fretting hand will naturally push the capo back and out of position quite easily when playing open chord tooim not sure why the plastic version ha this issue when it appears to have an identical design to the old metal one it doesnt seem defective a no part are bent or missing a far a i can tell unless i just happen to get one with a shorter than normal screw i wouldnt recommend it for any guitar without a thick neckmy recommendation is the classic kyser capo thats more dependable and quicker to apply and remove it also attache to your headstock effortlessly when not in use if you are looking for the screwdown style i would look for the older version of this and avoid the 34lite34,2,Will Not Clamp Tightly Enough
332,A22SZV0N4O2FQM,B0002CZTIO,i bought this pick holder because i hate putting a pick between my string to hold it and because you cant play with a pick in your string i originally thought this thing would be kind of large and go all the way across the headstock boy wa i wrong the picture is misleading this thing is tiny and barely go across three string it wedge under neath two string and you can put two pick maybe more if they are thin into the wedge on top stay in place well and is very convenientonly negative is i dont know how long it will hold up i think maybe a thicker rubber rather than this foam rubber would have held up better i can see this thing eventually losing tension in the pick holder wedge and dropping pick,4,Nice little pick holder
...,...,...,...,...,...
1360,A6GXP9ESXIH7O,B0002E1G5C,this winder and cutterwinds and cut finebut the pin puller doe not fit my guitar pin even though the ad claim that 34it fit virtually all guitar banjo and mandolins34 oh wellanother lessson learned,3,Good but not excellent...
5595,A2BB6IKKR4G40J,B000T517Y4,i am a tone freak i bought this pedal a while back under the impression that it would sound equal to a big muff at the time i had never owned a big muff i thought the sound it produced wa big muff sound i later took it to my friend house and matched setting with a real big muff the difference wa huge the big muff sounded fuller where the little one sounded like you just took the sound and

# Text Representation

In [47]:
#TF-IDF

#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer()


#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(X_train['reviewText'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

<700x6339 sparse matrix of type '<class 'numpy.float64'>'
	with 43905 stored elements in Compressed Sparse Row format>

In [48]:
#Bag of words for Text Representation

from sklearn.feature_extraction.text import CountVectorizer

CountVec = CountVectorizer()
#transform
Count_data = CountVec.fit_transform(X_train['reviewText'])

Count_data.shape

(700, 6339)

In [4]:
"""pip install gensim
pip install nltk"""

'pip install gensim\npip install nltk'

In [ ]:
"""from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec"""

In [ ]:
""""corpus_text = 'n'.join(rev[:1000]['reviewText'])
data = []
# iterate through each sentence in the file
for i in sent_tokenize(corpus_text):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [ ]:
""S""model1 = gensim.models.Word2Vec(data, min_count = 1,size = 100, window = 5, sg=0) 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5, sg = 1)"""